# 🎥 **04-Inferencia de Video con YOLO**

Este notebook aplica un modelo YOLO entrenado (o pre-entrenado) para detectar y contar vehículos en un video, guardar la secuencia anotada y reportar métricas de rendimiento (FPS, latencia, tiempo total).

In [1]:
from pathlib import Path
import cv2
import time
from ultralytics import YOLO
from collections import defaultdict

In [2]:
# Video de entrada 
VIDEO_IN = Path("/home/guardiaserver/bogota/vision-urbana-bogota/data/test/4K Road traffic video.mp4")

In [3]:
# Pesos del modelo YOLO entrenado 
MODEL_PT = Path("/home/guardiaserver/bogota/vision-urbana-bogota/models/yolov10m/weights/best.pt")

In [4]:
# Caperta de salida para los resultados
OUTPUT_DIR = Path("/home/guardiaserver/bogota/vision-urbana-bogota/results/video_inference")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [5]:
# Video de salida con las detecciones
VIDEO_OUT = OUTPUT_DIR / f"{VIDEO_IN.stem}_yolo.mp4"
# Otros parámetros
IMG_SIZE = 640
CONF_THRESH = 0.25
DEVICE = 0            # 0 → GPU 0,  
SHOW_FRAMES = False   # True para ver frames en tiempo de ejecución

In [6]:
model = YOLO(str(MODEL_PT))
model.fuse()  # acelera inferencia
print(f"Modelo {MODEL_PT.name}")

YOLOv10m summary (fused): 369 layers, 16,451,542 parameters, 0 gradients, 63.4 GFLOPs
Modelo best.pt


In [7]:
cap = cv2.VideoCapture(str(VIDEO_IN))
assert cap.isOpened(), f"No se pudo abrir {VIDEO_IN}"

# Propiedades del video
fps_in  = cap.get(cv2.CAP_PROP_FPS)
width   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Resolución: {width}×{height}  |  FPS original: {fps_in:.2f}")

# 🎥 VideoWriter para guardar la salida
fourcc = cv2.VideoWriter_fourcc(*"mp4v") # type: ignore
writer = cv2.VideoWriter(str(VIDEO_OUT), fourcc, fps_in, (width, height))

Resolución: 1920×1080  |  FPS original: 30.00


In [8]:
# 🚀 Inferencia y escritura de frames
frame_count, t0 = 0, time.time()
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Inferencia
    res = model.predict(
        frame,
        imgsz=IMG_SIZE,
        conf=CONF_THRESH,
        device=DEVICE,
        stream=False,
        verbose=False
    )[0]

    # Dibujar resultados sobre el frame
    annotated = res.plot()

    # Escribir frame anotado
    writer.write(annotated)

    # Mostrar en pantalla opcional
    if SHOW_FRAMES:
        cv2.imshow("YOLO Inference", annotated)
        if cv2.waitKey(1) & 0xFF == 27:   # ESC para salir
            break

    frame_count += 1

t_total = time.time() - t0
cap.release()
writer.release()
if SHOW_FRAMES:
    cv2.destroyAllWindows()

In [9]:
fps_real = frame_count / t_total
lat_ms   = 1000 * t_total / frame_count

print(f"📈  Frames procesados : {frame_count}")
print(f"⏱️  Tiempo total      : {t_total:.2f} s")
print(f"⚡ FPS promedio       : {fps_real:.2f}")
print(f"⌛ Latencia promedio  : {lat_ms:.2f} ms por frame")
print(f"🎞️  Video guardado en : {VIDEO_OUT}")

📈  Frames procesados : 9184
⏱️  Tiempo total      : 126.31 s
⚡ FPS promedio       : 72.71
⌛ Latencia promedio  : 13.75 ms por frame
🎞️  Video guardado en : /home/guardiaserver/bogota/vision-urbana-bogota/results/video_inference/4K Road traffic video_yolo.mp4
